# TensorFlow Keras Custom Layers and Models

- Create custom layers and integrate into a Keras model
- Compile, train, and evaluate the model

## 1 Import libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Softmax

## 2 Define a custom layer

Custom dense layer - 32 units, ReLU activation

A custom layer:
- Inheritated from Layer class
- 3 necessary methods:
  - \_\_init\_\_(self, units=?)    (#For structure)
  - build(self, input_shape)      (#For defining weights and biases)
    - weights & biases: add_weight(shape, initializer, trainable)
  - call(self, inputs)             (#For forward pass)

In [2]:
class CustomDenseLayer(Layer):

    # Create the layer with certain number of units
    def __init__(self, units=32):
        super(CustomDenseLayer, self).__init__()
        self.units = units

    # Create weights and biases
    # The shapes need to match for computation
    def build(self, input_shapes):
        # input_shapes will be the shape of the input tensor

        # Weights
        self.w = self.add_weight(shape=(input_shapes[-1], self.units),
                                 initializer='random_normal',  # Random normal initializer
                                 trainable=True)
        # Biases
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',      # Zero initializer
                                 trainable=True)
    # Forward pass
    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)  # ReLU activation


## 3 Work with the model

In [3]:
# Create the model

model = Sequential([
    CustomDenseLayer(128), # Input
    CustomDenseLayer(10),  # Hidden (ReLU)
    Softmax(),             # Output (Softmax - multi-class classification)
])

Considering the choice of activation: 

Output layer using Softmax:
- Type of task: Multi-class classification, so we have output probabilities summing up to 1 for each class
- Alignment with loss: Sum-to-one probabilities and Categorical cross-entropy


In [4]:
# Compile the model: before and after building

model.compile(loss='categorical_crossentropy',optimizer='adam')
print("Model summary before building:")
# The model doesn't know the shape yet
model.summary()

# Build the model, show parameters
model.build((1000, 20)) 
print("\nModel summary after building:")
model.summary()

Model summary before building:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ custom_dense_layer                   │ ?                           │     0 (unbuilt) │
│ (CustomDenseLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ custom_dense_layer_1                 │ ?                           │     0 (unbuilt) │
│ (CustomDenseLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax (Softmax)                    │ ?                           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ custom_dense_layer                   │ (1000, 128)                 │           2,688 │
│ (CustomDenseLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ custom_dense_layer_1                 │ (1000, 10)                  │           1,290 │
│ (CustomDenseLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax (Softmax)                    │ (1000, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,978 (15.54 KB)

 Trainable params: 3,978 (15.54 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Train the model

import numpy as np

x_train = np.random.random((1000, 20))

y_train = np.random.randint(10, size=(1000, 1))
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10) # Convert labels to one-hot encoding

model.fit(x_train, y_train,epochs=10, batch_size=32)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.3032   
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2983 
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.2969 
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.2998 
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.2954 
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2954 
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2947 
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2948 
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2923 
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2858 


In [6]:
# Evaluate the model

x_test = np.random.random((200, 20))
y_test = np.random.randint(10, size=(200, 1))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

loss = model.evaluate(x_test, y_test)
print(f"Test loss: {loss}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.3087  
Test loss: 2.307579517364502


## 4 Visualize Model Architecture

In [8]:
# Visualize the model architecture

from tensorflow.keras.utils import plot_model

plot_model(model, to_file="model_architecture_0.png", show_shapes=True, show_layer_names=True)



"dot" with args ['-Tps', 'C:\\Users\\rui-s\\AppData\\Local\\Temp\\tmp25ostmv7\\tmp5s8a3nu_'] returned code: 3221225477

stdout, stderr:
 b''
b''



AssertionError: "dot" with args ['-Tps', 'C:\\Users\\rui-s\\AppData\\Local\\Temp\\tmp25ostmv7\\tmp5s8a3nu_'] returned code: 3221225477

---

## Add Dropout layer to the model

In [ ]:
# Create the model

from tensorflow.keras.layers import Dropout

model = Sequential([
    CustomDenseLayer(128), # Input
    Dropout(rate=0.5),     # Dropout layer
    CustomDenseLayer(10),  # Hidden (ReLU)
    Softmax(),             # Output (Softmax - multi-class classification)
])


# Recompile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model, again
model.fit(x_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.3007   
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2987 
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2940 
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2957 
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2854 
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2851 
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2928 
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2804 
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2810 
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2740 


## Adjust the number of units in Custom Layer

In [ ]:
class CustomDenseLayer(Layer):
    
    def __init__(self, units=128):
        super(CustomDenseLayer, self).__init__()
        self.uits = units
    
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True                                 
                                 )
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',
                                 trainable=True)
        
    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)
    
model = Sequential([
    CustomDenseLayer(128),
    CustomDenseLayer(10)
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(x_train, y_train, epochs=10, batch_size=32)

model.evaluate(x_test, y_test)
    

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: None (of type <class 'NoneType'>)